# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [70]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [71]:
X = df.to_numpy()
X = np.delete(X, X.shape[1] - 1, 1)
Y = df["quality"]

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [75]:
train_end_index = int(X.shape[0] * 0.7)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [76]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
train_predictions = linear_model.predict(X_train)
test_predictions = linear_model.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [77]:
print("Train MSE: " + str(mean_squared_error(y_train, train_predictions)))
print("Test MSE: " + str(mean_squared_error(y_test, test_predictions)) + "\n")

print("Train R-squared: " + str(r2_score(y_train, train_predictions)))
print("Test R-squared: " + str(r2_score(y_test, test_predictions)))

Train MSE: 0.4014418059351661
Test MSE: 0.4574562438018577

Train R-squared: 0.3863564776897399
Test R-squared: 0.29186625432885205


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [78]:
lm_folds = LinearRegression()
print("Mean R-squared: " + str(cross_val_score(lm_folds, X_train, y_train, scoring = 'r2', cv = 5).mean()))

Mean R-squared: 0.3588882591006284


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [79]:
from sklearn.linear_model import Lasso

lm_lasso = Lasso()
# Первая колонка для альфа, вторая для соответствуеющего среднего R-квадрата
alpha_score = [[], []]
for alpha in np.arange(0.1, 1.1, 0.1):
    lm_lasso.alpha = alpha
    lm_lasso.fit(X_train, y_train)
    y_pr = lm_lasso.predict(X_test)
    R2 = r2_score(y_test, y_pr)
    #R2 = cross_val_score(lm_lasso, X_train, y_train, scoring = "r2", cv = 5).mean()
    alpha_score[0].append(alpha)
    alpha_score[1].append(R2)

In [80]:
index = alpha_score[1].index(max(alpha_score[1]))
print("Best fitting alpha is: " + str(alpha_score[0][index]))

Best fitting alpha is: 0.1


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [81]:
alphas = { "alpha" : np.arange(0.1, 1.1, 0.1) }

grid = GridSearchCV(lm_lasso, alphas)
grid.fit(X_train, y_train)

print("Lasso best score: " + str(grid.best_score_))
print("Lasso best parametres: " + str(grid.best_estimator_))

Lasso best score: 0.23575420465743582
Lasso best parametres: Lasso(alpha=0.1)


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [82]:
lm_lasso = Lasso(alpha = 0.1).fit(X_train, y_train)
y_pred = lm_lasso.predict(X_test)
print("R-squared: " + str(r2_score(y_test, y_pred)))

R-squared: 0.2389545415904254


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [83]:
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([("poly", PolynomialFeatures(degree = 2)), 
                ("regression", LinearRegression())])
cross_val_score(pipe, X_train, y_train, scoring = "r2", cv = 5).mean()

0.2605270902583659

### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [85]:
pipe.fit(X_train, y_train)
pipe_train_pred = pipe.predict(X_train)
pipe_test_pred = pipe.predict(X_test)

print("Train MSE: " + str(mean_squared_error(y_train, pipe_train_pred)))
print("Test MSE: " + str(mean_squared_error(y_test, pipe_test_pred)) + "\n")

print("Train R-squared: " + str(r2_score(y_train, pipe_train_pred)))
print("Test R-squared: " + str(r2_score(y_test, pipe_test_pred)))

Train MSE: 0.3581171062826742
Test MSE: 0.4906296613339625

Train R-squared: 0.4525825680090001
Test R-squared: 0.24051442181588878


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

**Ответ**
1. Исходная модель показала не очень высокое качество, переобучения не было
2. Регуляризация не смогла улучшить качество модели
3. Добавление полиномов значительно улучшило качество, новые признаки снизили переобучение

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [104]:
new_pipeline = Pipeline([("poly", PolynomialFeatures(degree = 1)), 
                         ("regression", LinearRegression())])

print("With cross-validation: " + str(cross_val_score(new_pipeline, X_train, y_train, scoring = "r2", cv = 5).mean()) + "\n")

new_pipeline.fit(X_train, y_train)
train_pred = new_pipeline.predict(X_train)
test_pred = new_pipeline.predict(X_test)

print("Train MSE: " + str(mean_squared_error(y_train, train_pred)))
print("Test MSE: " + str(mean_squared_error(y_test, test_pred)) + "\n")

print("Train R-squared: " + str(r2_score(y_train, train_pred)))
print("Test R-squared: " + str(r2_score(y_test, test_pred)))

With cross-validation: 0.3588882591006276

Train MSE: 0.4014418059351661
Test MSE: 0.4574562438018578

Train R-squared: 0.3863564776897399
Test R-squared: 0.2918662543288518


## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [2]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [5]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


#### Способ 2

In [6]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [9]:
from sklearn.model_selection import KFold

print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])
kf = KFold(5, shuffle = False)
s = 0
for train_index, test_index in kf.split(all_data):
    mean = all_data.iloc[train_index].groupby('item_id').target.mean()
    all_data.iloc[test_index].item_target_enc = all_data.iloc[test_index].item_id.map(mean)
encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4830386988621791
0.48303869886217926


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [10]:
target_sum = all_data.target.sum()

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

3648206.0
0.48303869886217926


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [ ]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4830386988621791


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [ ]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4830386988621791


Ожидаемый ответ 0.5025